In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.layers import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
import datetime
import time

In [ ]:
!gsutil cp gs://gtsrb_raw_final/GTSRB_Final_Training_Images.zip .
!unzip GTSRB_Final_Training_Images.zip

In [2]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='training')

val_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='validation')

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [3]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(100, 100, 3),
    classifier_activation="softmax",
)

2022-05-08 03:06:18.587967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 03:06:18.598720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 03:06:18.599469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 03:06:18.600548: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
model = models.Sequential()
model.add(xception)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=43, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dropout (Dropout)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 43)                792619    
                                                                 
Total params: 21,654,099
Trainable params: 21,599,571
Non-trainable params: 54,528
_________________________________________________________________


In [6]:
%load_ext tensorboard

In [7]:
log_dir = "logs_xception/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
steps_per_epoch = 800

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [12]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [13]:
tic = time.time()
model.fit(train_generator, validation_data=val_generator, epochs = 20, callbacks=[es,tensorboard_callback])
toc = time.time()

Epoch 1/20
491/491 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 0.9858

2022-05-08 03:19:06.786550: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 754974720 exceeds 10% of free system memory.


491/491 [==============================] - 142s 282ms/step - loss: 0.0029 - accuracy: 0.9858 - val_loss: 0.0056 - val_accuracy: 0.9702
Epoch 2/20
491/491 [==============================] - 137s 280ms/step - loss: 0.0015 - accuracy: 0.9932 - val_loss: 0.0045 - val_accuracy: 0.9767
Epoch 3/20
491/491 [==============================] - 138s 281ms/step - loss: 0.0019 - accuracy: 0.9909 - val_loss: 0.0025 - val_accuracy: 0.9861
Epoch 4/20
491/491 [==============================] - 138s 280ms/step - loss: 0.0015 - accuracy: 0.9935 - val_loss: 0.0858 - val_accuracy: 0.8792
Epoch 5/20
491/491 [==============================] - 137s 278ms/step - loss: 0.0010 - accuracy: 0.9947 - val_loss: 0.0038 - val_accuracy: 0.9777
Epoch 6/20
491/491 [==============================] - 137s 280ms/step - loss: 6.3832e-04 - accuracy: 0.9963 - val_loss: 0.0062 - val_accuracy: 0.9732
Epoch 7/20
491/491 [==============================] - 137s 279ms/step - loss: 0.0012 - accuracy: 0.9946 - val_loss: 0.0017 - val_ac

In [ ]:
print('Time taken for training: {} '.format(toc-tic))

In [ ]:
%tensorboard --logdir logs_xception/fit

In [129]:
for layer in xception.layers:
    layer.trainable = False
model1 = models.Sequential()
model1.add(xception)
model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(units=43, activation='softmax'))
model1.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 flatten_11 (Flatten)        (None, 18432)             0         
                                                                 
 dropout_11 (Dropout)        (None, 18432)             0         
                                                                 
 dense_11 (Dense)            (None, 43)                792619    
                                                                 
Total params: 21,654,099
Trainable params: 792,619
Non-trainable params: 20,861,480
_________________________________________________________________


In [130]:
log_dir = "logs_xception_NT/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [131]:
steps_per_epoch = 729

optimizer = tf.keras.optimizers.Adam()

model1.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [132]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [133]:
tic = time.time()
model1.fit(train_generator, validation_data=val_generator, epochs = 10, callbacks=[es,tensorboard_callback])
toc = time.time()

Epoch 1/10
491/491 [==============================] - 93s 186ms/step - loss: 0.0046 - accuracy: 0.9909 - val_loss: 0.0038 - val_accuracy: 0.9846
Epoch 2/10
491/491 [==============================] - 91s 185ms/step - loss: 3.9135e-04 - accuracy: 0.9996 - val_loss: 0.0028 - val_accuracy: 0.9847
Epoch 3/10
491/491 [==============================] - 90s 183ms/step - loss: 2.3744e-04 - accuracy: 0.9995 - val_loss: 0.0026 - val_accuracy: 0.9833
Epoch 4/10
491/491 [==============================] - 89s 182ms/step - loss: 1.7529e-04 - accuracy: 0.9996 - val_loss: 0.0025 - val_accuracy: 0.9833
Epoch 5/10
491/491 [==============================] - 89s 182ms/step - loss: 1.2429e-04 - accuracy: 0.9997 - val_loss: 0.0026 - val_accuracy: 0.9824
Epoch 6/10
491/491 [==============================] - 90s 183ms/step - loss: 1.4033e-04 - accuracy: 0.9995 - val_loss: 0.0023 - val_accuracy: 0.9838
Epoch 7/10
491/491 [==============================] - 89s 181ms/step - loss: 9.1148e-05 - accuracy: 0.9998 - v

In [134]:
print('Time taken for training: {} '.format(toc-tic))

Time taken for training: 900.1436853408813 


In [ ]:
%tensorboard --logdir logs_xception_NT/fit